In [1]:
import numpy as np
import nltk as nl
from gensim.models import Word2Vec
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from nltk.corpus import stopwords
import string
from collections import Counter
from gensim.parsing.porter import PorterStemmer
from gensim.utils import lemmatize


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

Using TensorFlow backend.


In [3]:
surveys = pd.read_excel("C:\\Machine Learning\\Hackathons\\Xerox\\customer_Care_Survey_Refined.XLSX",header=0,sheet_name="Sheet2")

In [14]:
surveys.shape

(2315, 31)

In [15]:
surveys.head()

,Month,RespondentID,CollectorID,StartDate,EndDate,IP Address,Email Address,First Name,LastName,Custom Data,...,REASON_CODE,ROUTING_CODE,CLOSE_DATE,ROOTCAUSE,Rep Name,Mgr Name,Disposition,eMail,Count1,Count2
0,201812,5576030682,39775725,2018-12-31 14:41:01,2018-12-31 14:42:00,13.13.16.1,NaN,NaN,NaN,NaN,...,Equip Billing Dates,Meters,2018-12-31 15:08:42,Customer Explanation,"Manzo Navas, Jose Alejandro",Jose Rubio,Resolution,na@na.com,NaN,NaN
1,201812,5576020478,39775725,2018-12-31 13:29:57,2018-12-31 13:30:27,13.8.125.11,NaN,NaN,NaN,NaN,...,Equip Billing Pricing,Base,2018-12-31 10:36:25,OTI Edit Defect / VQUIX Issue,"Mancil, David",Elizabeth Campbell-Moore,Resolution,eriban.arenales@xerox.com,NaN,NaN
2,201812,5576004947,39775725,2018-12-31 11:52:58,2018-12-31 11:57:57,96.61.151.223,NaN,NaN,NaN,NaN,...,Late Charges,Late Charge Incorrectly Billed,2018-12-31 13:51:43,NaN,"Revolorio, Yanira",Mario Pineda,POC,pam@windsormountain.org,NaN,NaN
3,201812,5575981269,39775725,2018-12-31 09:47:29,2018-12-31 09:48:44,173.161.54.97,NaN,NaN,NaN,NaN,...,Web Issues,EIPP Issue,2018-12-26 14:10:52,Other,"Pinot, Stephanie",Maria Acevedo,Resolution,renee@busybeaver.net,NaN,NaN
4,201812,5575964427,39775725,2018-12-31 08:17:20,2018-12-31 08:17:58,172.100.193.136,NaN,NaN,NaN,NaN,...,Equip Performance,Downtime Credit,2018-12-26 09:32:29,Equipment Issue,"Ramirez Moreira, Cesar Mario",Jose Rubio,Resolution,keith.koster@xerox.com,NaN,NaN


In [6]:
surveys_comments = pd.DataFrame({"SR#":surveys["SR#"],
                                 "Comments":surveys["Do you have any other details you want to share regarding the ha"],
                                 "Additional_Comments":surveys["Please provide details#"],
                                 "Full_Comments":surveys["Full_Comments"],
                                 "Customer_SAT_Score":surveys["How satisfied are you with the way your issue was resolved?"]})

In [7]:
surveys_comments.shape

(2315, 5)

In [8]:
surveys_comments.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
0,1-101526171125,Many thanks for understanding this issue and r...,NaN,Many thanks for understanding this issue and r...,10
1,1-101320199441,Thank you for everything David!,NaN,Thank you for everything David!,10
2,1-101571535327,We have been a customer of yours for many many...,No one wants to resolve this issue. Everyone ...,We have been a customer of yours for many many...,1
3,1-101428425708,It doesn't feel like a person helped me at all...,I needed information on a recurring bill we're...,It doesn't feel like a person helped me at all...,1
4,1-101292413714,none,NaN,none,10


In [9]:
surveys_comments["Full_Comments"][3], surveys_comments["Customer_SAT_Score"][3]

("It doesn't feel like a person helped me at all.  It felt like an automated response. I needed information on a recurring bill we're paying and did not receive an answer.  Instead, I was pointed to log into the website to review - but it only has 1 of 2 of our bills on the website portal.",
 1)

In [20]:
surveys_comments.groupby(["Customer_SAT_Score"])["Customer_SAT_Score"].count()

Customer_SAT_Score
0       72
1      792
2       85
3       39
4       22
5       65
6       24
7       36
8       70
9      107
10    1003
Name: Customer_SAT_Score, dtype: int64

In [21]:
surveys_comments.loc[surveys_comments["Customer_SAT_Score"]<8,"Customer_SAT_Score"] = 1
surveys_comments.loc[surveys_comments["Customer_SAT_Score"]>=8,"Customer_SAT_Score"] = 0

In [22]:
surveys_comments.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
0,1-101526171125,Many thanks for understanding this issue and r...,NaN,Many thanks for understanding this issue and r...,0
1,1-101320199441,Thank you for everything David!,NaN,Thank you for everything David!,0
2,1-101571535327,We have been a customer of yours for many many...,No one wants to resolve this issue. Everyone ...,We have been a customer of yours for many many...,1
3,1-101428425708,It doesn't feel like a person helped me at all...,I needed information on a recurring bill we're...,It doesn't feel like a person helped me at all...,1
4,1-101292413714,none,NaN,none,0


In [23]:
len(surveys_comments)

2315

In [24]:
surveys_comments.loc[1]

SR#                                       1-101320199441
Comments                Thank you for everything David! 
Additional_Comments                                  NaN
Full_Comments          Thank you for everything David!  
Customer_SAT_Score                                     0
Name: 1, dtype: object

## K-mean clustering of text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [ ]:
vocab = []
for i in range(10):
    tokens = clean_doc(surveys_comments["Full_Comments"].iloc[i])
    #print(tokens)
    vocab.append(tokens)
    #docss= clean_doc(surveys_comments["Full_Comments"])

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
XX = vectorizer.fit_transform(surveys_comments["Full_Comments"])

In [ ]:
xx = CountVectorizer(lowercase=True)

In [ ]:
true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=50, n_init=3)
model.fit(XX)

In [ ]:
model.inertia_

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
order_centroids

In [ ]:
for i in range(true_k):
     print("Cluster %d:" % i),
     for ind in order_centroids[i, :11]: # change 11 to see less or more cluster words
         print("%s" % terms[ind])

## end Clustering

In [26]:
from sklearn.model_selection import train_test_split
import os
from nltk.corpus import stopwords
import string
from collections import Counter

In [27]:
trainx_outer, testx_outer,trainy_outer, testy_outer = train_test_split(surveys_comments,
                                                                       surveys_comments["Customer_SAT_Score"],test_size=0.30,random_state =10,
                                                                       stratify = surveys_comments["Customer_SAT_Score"])


In [28]:
trainx_outer.shape, testx_outer.shape,trainy_outer.shape, testy_outer.shape

((1620, 5), (695, 5), (1620,), (695,))

In [29]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    #print("table values are :\n")
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [30]:
trainx_outer.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
740,1-99756021787,NaN,I requested instructions from Xerox to provide...,I requested instructions from Xerox to provid...,1
1917,1-95817864576,Lynne Jones is awesome :),NaN,Lynne Jones is awesome :),0
12,1-101402551768,NaN,I was pushed off to another rep that I have be...,I was pushed off to another rep that I have b...,1
234,1-100776372013,NaN,we have never had a bill this high and we trie...,we have never had a bill this high and we tri...,1
1458,1-97805369279,Great follow up and knowledge sharing!,NaN,Great follow up and knowledge sharing!,0


In [31]:
testx_outer.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
928,1-99323730564,No. She was professional and found my duplicat...,NaN,No. She was professional and found my duplicat...,0
622,1-100033598619,Darlene did a very good job of making sure we ...,NaN,Darlene did a very good job of making sure we ...,0
287,1-100687070975,this has been going on for far to long. I hav...,I have been going around in circles with this ...,this has been going on for far to long. I hav...,1
1452,1-97826397517,I got what I asked for in a surprisingly short...,NaN,I got what I asked for in a surprisingly short...,0
1746,1-96179624984,Karla Sandoval did a good job of resolving thi...,NaN,Karla Sandoval did a good job of resolving thi...,0


In [32]:
trainx_outer.iloc[3,3]

' we have never had a bill this high and we tried to reach out to Carlos by phone at least 5 time and via email 1, but did not get a respond back'

In [33]:
vocab = Counter()
for i in range(len(trainx_outer)):
    #pass the Full_comment column for each row in the train_outer
    tokens = clean_doc(trainx_outer.iloc[i,3])
    vocab.update(tokens)

In [ ]:
vocab

In [ ]:
vocab.items()

In [34]:
min_occurance = 2
tokens = [k for k,c in vocab.items() if c >= min_occurance]
print(len(tokens))

2355


In [35]:
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()
save_list(tokens, 'vocab_CCO.txt')

In [36]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [37]:
# load the vocabulary
vocab_filename = 'vocab_CCO.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))

2355


In [38]:
def clean_doc_emb(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

In [39]:
records = list()
for i in range(len(trainx_outer)):
    tokens = clean_doc_emb(trainx_outer["Full_Comments"].iloc[i], vocab)
    records.append(tokens)

In [ ]:
records

In [40]:
train_docs = records
train_docs

['requested instructions Xerox provide customer needs return unused supplies received email back email template updated Customer Care representative',
 'Lynne Jones awesome',
 'pushed another rep unable get Also dont know name Mary came one business name',
 'never bill high tried reach Carlos phone least time via email get respond back',
 'Great follow knowledge',
 'Return instructions little bit done',
 'went beyond provide requested thank Hector',
 'worked Xerox close years The difficulty company experiences like gentleman wonder company Great working',
 'Dont know sure yet quoted price buyout told different price sent check price first quoted see resolved',
 'reached get detailed call log communications customer service department notified one call listed log records show least documented calls end Please advise possible',
 'Marizabel took ownership right away case worked priority Really appreciated really appreciate got completed ASAP Thank Marizabel',
 'heard back anyone Still wai

In [41]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

In [42]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)
#tokenizer.fit_on_texts(records)

In [43]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)
#encoded_docs = tokenizer.texts_to_sequences(records)

In [44]:
# pad sequences
max_length = max([len(s.split()) for s in records])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [45]:
Xtrain.shape

(1620, 252)

In [46]:
test_docs = list()
for i in range(len(testx_outer)):
    tokens = clean_doc_emb(testx_outer["Full_Comments"].iloc[i], vocab)
    test_docs.append(tokens)

In [47]:
encoded_docs_test = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [48]:
Xtest.shape

(695, 252)

In [49]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

In [64]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
len(vocab)

## Pre Trained Model

In [51]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [52]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

In [53]:
os.chdir("C:\\Machine Learning\\Hackathons\\Sample Data\\NLP\\Sentiment-Analysis\\txt_sentoken\\")

In [54]:
from numpy import asarray
from numpy import zeros

In [55]:
vocab_size = len(vocab) + 1

In [65]:
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [66]:
embedding_layer.get_weights()

[]

In [58]:
from keras.layers import LSTM,Bidirectional,Dropout, RNN, GRU

In [59]:
Xtrain.shape

(1620, 252)

### Choosing the right amount of nodes and layers
There is no final, definite, rule of thumb on how many nodes (or hidden neurons) or how many layers one should choose, and very often a trial and error approach will give you the best results for your individual problem. The most common framework for this is most likely the k-fold cross-validation. However, even for a testing procedure, we need to choose some (k) numbers of nodes.  
The following formula may give you a starting point:    

  N_s/(N_i+N_o)  
  
Nᵢ is the number of input neurons, Nₒ the number of output neurons, Nₛ the number of samples in the training data, and α represents a scaling factor that is usually between 2 and 10. We can calculate 8 different numbers to feed into our validation procedure and find the optimal model, based on the resulting validation loss.
If the problem is simple and time an issue, there are various other rules of thumbs to determine the number of nodes, which are mostly simply based on the input and output neurons. We have to keep in mind that, while easy to use, they will rarely yield the optimal result. Here is just one example, which we will use for this basic model:  
  
2/3*(N_i+N_o)  

In [67]:
# define model
model = Sequential()
model.add(embedding_layer)
#model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128))
model.add(Dropout(0.5))
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 252, 100)          201800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 319,177
Trainable params: 117,377
Non-trainable params: 201,800
_________________________________________________________________
None


In [68]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, trainy_outer, epochs=14, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, testy_outer, verbose=0)
print('Test Accuracy: {} and loss is {}' .format(acc*100,loss))

Epoch 1/14
 - 13s - loss: 0.6935 - acc: 0.4846
Epoch 2/14
 - 13s - loss: 0.6932 - acc: 0.5099
Epoch 3/14
 - 12s - loss: 0.6927 - acc: 0.5025
Epoch 4/14
 - 12s - loss: 0.6928 - acc: 0.5093
Epoch 5/14
 - 12s - loss: 0.6930 - acc: 0.5000
Epoch 6/14
 - 12s - loss: 0.6930 - acc: 0.4951
Epoch 7/14
 - 12s - loss: 0.6931 - acc: 0.5049
Epoch 8/14
 - 12s - loss: 0.6932 - acc: 0.5086
Epoch 9/14
 - 13s - loss: 0.7318 - acc: 0.5062
Epoch 10/14
 - 11s - loss: 0.7156 - acc: 0.5074
Epoch 11/14
 - 11s - loss: 0.6983 - acc: 0.5074
Epoch 12/14
 - 11s - loss: 0.7060 - acc: 0.4784
Epoch 13/14
 - 11s - loss: 0.7041 - acc: 0.4975
Epoch 14/14
 - 12s - loss: 0.6972 - acc: 0.4969
Test Accuracy: 44.316546779742346 and loss is 0.7025183118504585


Test Accuracy: 83.45323731573366 and loss is 0.5199229238273428 LSTM
Test Accuracy: 85.17985602076963 and loss is 0.4345971986544218 GRU
Test Accuracy: 82.01438839487034 and loss is 0.5317774833106309 GRU
Test Accuracy: 81.43884886082985 and loss is 0.5419978845891334

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
history = model.fit(Xtrain, trainy_outer, validation_split=0.25, epochs=4, verbose=2)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_predict = model.predict(Xtest)
y_predict[:10]

In [ ]:
y_predict[y_predict>=.5] = 1
y_predict[y_predict<.5] = 0

In [ ]:
y_predict.shape

In [ ]:
#final_comp =  pd.DataFrame(list(testy_outer), columns=["Actual"],)
#print(final_comp.shape)
y_predict =   pd.DataFrame(y_predict, columns=["Predicted"],)
print(y_predict.shape)
#print(testx_outer.shape)
y_predict.head(10)

In [ ]:
test_temp = testx_outer
test_temp.shape, testx_outer.shape

In [ ]:
temp = test_temp
temp.reset_index(inplace=True,drop=True)
#temp.head()
act_vs_pred =pd.concat([ temp,y_predict], axis=1,)

In [ ]:
#act_vs_pred.drop(columns=["index"],inplace=True)

In [ ]:
act_vs_pred

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
mat = confusion_matrix(act_vs_pred["Customer_SAT_Score"], act_vs_pred["Predicted"])

In [ ]:
mat = pd.DataFrame(data=mat, columns=["Actual Positive","Actual Negative"],
                   index = ["Predicted Positive","Predicted Negative"])

In [ ]:
mat

In [ ]:
t = act_vs_pred.copy()

In [ ]:
t.head(3)

In [ ]:
t["Mismatch"] =  False

In [ ]:
t.loc[t["Customer_SAT_Score"]!= t["Predicted"],"Mismatch"] = True

In [ ]:
t

In [ ]:
t.loc[9]

## Sequencing Modeling